In [1]:
# !pip install -i https://artifactory.global.standardchartered.com/artifactory/api/pypi/pypi/simple psycopg2-binary
import csv
import requests
import psycopg2
from pytz import timezone
from datetime import datetime, timedelta
from dateutil import parser, relativedelta
import json

START_DATE = "2022-09-01";
END_DATE = "2022-09-30";
POSTGRES_USER = "SE_MOSAI"
POSTGRES_PASSWORD = 'MkoNgt#28#asd'
POSTGRES_HOST = "hklvdpsql062.hk.standardchartered.com"
POSTGRES_PORT = "6524"
POSTGRES_DB_CONSOLE = "mosaic_console"
POSTGRES_DB_AI = "ai_logistics"
POSTGRES_DB_METERING = "metering"
POSTGRES_DB_USERMGM = "usermgm"
POSTGRES_DB_MONITOR = "mosaic_monitor"
SERVER_URL="http://keycloak.keycloak/auth/"
REALM="master"
CLIENT_ID="admin-cli"
KEYCLOAK_USERNAME="keycloak"
KEYCLOAK_PASSWORD="90f8372d933d4fd1fd311f0b8cd36a3d"
APP_ADMIN_USER = "1632997"
TIME_ZONE_GMT = "Etc/GMT+7"
TIME_ZONE_IST = "Asia/Kolkata"


def get_project_data_with_users_details(connection):
    try:
        cursor = connection.cursor()
        postgreSQL_select_Query = "select id, project_name, description, created_by, created_date from "+POSTGRES_DB_CONSOLE+".project where (status='ACTIVE') AND (date(created_date) >= '"+START_DATE+"' AND date(created_date) <= '"+END_DATE+"')"
        cursor.execute(postgreSQL_select_Query)
        project_details_list = cursor.fetchall()
        return project_details_list
    except (Exception, psycopg2.Error) as error:
        print("Error while fetching data from PostgreSQL : ", error)
        return -1
    finally:
        cursor.close()

In [2]:
! pip freeze | grep psycop

psycopg2-binary==2.9.3


In [3]:
try:
    report_records = []
    connection_mosaic_console = psycopg2.connect(user=POSTGRES_USER,
                                                 password=POSTGRES_PASSWORD,
                                                 host=POSTGRES_HOST,
                                                 port=POSTGRES_PORT,
                                                 database=POSTGRES_DB_CONSOLE)
    project_details_list = get_project_data_with_users_details(connection_mosaic_console)
    print("The total projects are : ", len(project_details_list))
    
    for project in project_details_list:
        report_record = {}
        report_record["PROJECT_ID"] = project[0]
        report_record["PROJECT_NAME"] = project[1]
        report_record["PROJECT_DESCRIPTION"] = project[2]
        report_record["PROJECT_CREATED_BY"] = project[3]
        report_record["PROJECT_CREATED_DATE"] = project[4].astimezone(timezone(TIME_ZONE_IST)).strftime('%Y-%m-%d %H:%M:%S.%f')
        report_records.append(report_record)
        
    if report_records:
        with open('/data/project_details_v6_prod.csv', 'w', newline='') as output_file:
            dict_writer = csv.DictWriter(output_file, report_records[0].keys())
            dict_writer.writeheader()
            dict_writer.writerows(report_records)
    
    print("DONE !!")
except Exception as ex:
      print("Exception : ", ex)
finally:
    if connection_mosaic_console:
        connection_mosaic_console.close()
        print("\nPostgres Connection 'connection_mosaic_console' is closed") 


The total projects are :  30
DONE !!

Postgres Connection 'connection_mosaic_console' is closed
